In [ ]:
# Small example of network outputs

# Global import
import sys
sys.path.insert(0, '../pre_built')
import os
import torch
import torch.nn as nn
# import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
from collections import OrderedDict
from dataset import *
import sys
from train import *
from lr_analyzer import *
from criterion import *
from Unet import UNet
from dataset import *


In [ ]:
# Small example of network outputs

# Global import
import sys
import os
import torch
import torch.nn as nn
# import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
from collections import OrderedDict
from dataset import *
import sys
sys.path.insert(0, '../pytorch/')
from train import *
from lr_analyzer import *
from criterion import *

from Unet import UNet
from dataset import *

# Small example of network outputs

# Global import
import sys
import os
import torch
import torch.nn as nn
# import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
from collections import OrderedDict
from dataset import *
import sys
from train import *
from lr_analyzer import *
from criterion import *
import optuna
from optuna.trial import TrialState
# Local import

print(sys.path)
from Unet import UNet

from dataset import *
data_dir = 'Data1/'
# data_dir = 'Training/'
SetSz = len(os.listdir(data_dir))
print(SetSz )
from train import *

                                                           
import os
import time
import utils
import json
import logging
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
from dataset import psf_dataset, splitDataLoader, ToTensor, Normalize
from utils_visdom import VisdomWebServer
import aotools
from criterion import *

def objective(trial):
    """Objective function to be optimized by Optuna.

    Hyperparameters chosen to be optimized: optimizer, learning rate,
    dropout values, number of convolutional layers, number of filters of
    convolutional layers, number of neurons of fully connected layers.

    Inputs:
        - trial (optuna.trial._trial.Trial): Optuna trial
    Returns:
        - accuracy(torch.Tensor): The test accuracy. Parameter to be maximized.
    """
    model_dir='Optuna'
    os.system('rm -r '+model_dir)
    os.system('mkdir '+model_dir)
    data_dir = '../generation/Data1/'
    # data_dir = 'Training/'
    SetSz =64 #len(os.listdir(data_dir))
    dataset_size = SetSz
    dataset = psf_dataset(root_dir = data_dir, 
                      size = dataset_size,
                      transform = transforms.Compose([Noise(), Normalize(), ToTensor()]))
    criterion = RMSELoss()
#     batch = trial.suggest_int("batch", int(SetSz/2), 2*SetSz)  # Optimizers
    batch=64
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "SGD"])  # Optimizers

    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)     # Learning rates
    split = trial.suggest_float("split", 0.7, 0.99)     # Learning rates
    stpsz = trial.suggest_int("stpsz", 80, 170)     # Learning rates
    gmma = trial.suggest_float("gmma",0.1, 0.9)     # Learning rates
    model = UNet(2, 1)
    if optimizer_name=="Adam":
         optimizer = optim.Adam(model.parameters(), lr=lr)
    if optimizer_name=="SGD":
         mom = trial.suggest_float("mom", 0.1, 0.99)     # Learning rates
    if optimizer_name=="SGD":
         mom = trial.suggest_float("mom", 0.1, 0.99)     # Learning rates
         optimizer = optim.SGD(model.parameters(), lr=lr, momentum=mom)

    # Generate the model

    # Generate the optimizers

    

    # Training of the model
    for epoch in range(n_epochs):
        model, accuracy=train(model, model_dir, dataset, optimizer, criterion, split=[split, 1-split], batch_size=32, stpsz=stpsz, gmma=gmma, epoch=epoch) # Train the model

        # For pruning (stops trial early if not promising)
        trial.report(accuracy, epoch)
        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return accuracy
    
    
def clbk(Study, trial):  
    df = study.trials_dataframe().drop(['datetime_start', 'datetime_complete', 'duration'], axis=1)  # Exclude columns
    df = df.loc[df['state'] == 'COMPLETE']        # Keep only results that did not prune
    df = df.drop('state', axis=1)                 # Exclude state column
    df = df.sort_values('value')                  # Sort based on accuracy
    df.to_csv('optuna_resultsNew.csv', index=False)  # Save to csv file
    if trial.number%5==0:
        obj=plot_optimization_history(study)

        # plt.savefig('OptHist.png')

        pi.write_image(obj, 'OptHist2.png')

        obj1=plot_slice(study)
        pi.write_image(obj1, 'OpSlice2.png')

        obj2=plot_contour(study)
        pi.write_image(obj2, 'OptCont2.png')

         
    
def train(model, model_dir, dataset, optimizer, criterion, split=[0.9, 0.1], batch_size=32, stpsz=150, gmma=0.1, epoch=0):

#     # Logging
#     log_path = os.path.join(model_dir, 'logs.log')
#     os.system('nano '+log_path)
#     utils.set_logger(log_path)


    # ---
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=stpsz, gamma=gmma)
    metrics = {
        'train_loss': [],
        'val_loss': [],

    }
    # Device
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
   
    # Training
    since = time.time()
    dataset_size = {
        'train':int(split[0]*len(dataset)),
        'val':int(split[1]*len(dataset))
    }

    # Dataset
    dataloaders = {}
    dataloaders['train'], dataloaders['val'] = splitDataLoader(dataset, split=split, 
                                                             batch_size=batch_size, random_seed=random_seed)
    
    
    best_loss = 0.0

    model.train()  # Set model to training mode
    running_loss = 0.0
    zernike_loss =0.0
    phase='train'
    # Each epoch has a training and validation phase
    for phase in ['train', 'val']:
        if phase == 'train':
                model.train()  # Set model to training mode
        else:
                model.eval()   # Set model to evaluate mode
            
        for _, sample in enumerate(dataloaders[phase]):
              
                # GPU support 
                inputs = sample['image'].to(device)
                phase_0 = sample['phase'].to(device)

                if (not inputs[0].isnan().any()):

                   # zero the parameter gradients
                   optimizer.zero_grad()
                   #logging.info(' individual loss: %f %f' % (phase_0, phase_estimation))
                   # forward: track history if only in train
                   with torch.set_grad_enabled(phase == 'train'):

                       # Network return phase and zernike coeffs
                       phase_estimation = model(inputs)
                       rgg=torch.squeeze(phase_estimation)
                       loss = criterion(torch.squeeze(phase_estimation), phase_0)
                       if phase == 'train':
                            loss.backward()
                            optimizer.step()   
                       if loss.item()!=np.nan:
                             running_loss += 1 * loss.item() * inputs[0].size(0)    
#                        logging.info(' individual loss: %f %f' % (loss.item(), 
#                                                                  .size(0)))
                metrics[phase+'_loss'].append(running_loss / dataset_size[phase])
                if phase == 'val':
                   scheduler.step()
                   # Save weigths
                   if epoch == 0 or running_loss < best_loss:
                        best_loss = running_loss
                        model_path = os.path.join(model_dir, 'model.pth')
                        torch.save(model.state_dict(), model_path)
#     print(np.sum(metrics['val_loss']+metrics['train_loss']))
    return model, np.sum(metrics['val_loss']+metrics['train_loss'])

            
        
def get_lr(optimizer):
    for p in optimizer.param_groups:
        lr = p['lr']
    return lr                                           
                                                           
                                                           
n_epochs = 10                         # Number of training epochs
batch_size_train = 32                 # Batch size for training data
batch_size_test = 32                # Batch size for testing data
number_of_trials = 200                # Number of Optuna trials
limit_obs = True                      # Limit number of observations for faster computation


if limit_obs:  # Limit number of observations
        number_of_train_examples = 500 * batch_size_train  # Max train observations
        number_of_test_examples = 5 * batch_size_test      # Max test observations
else:
        number_of_train_examples = 60000                   # Max train observations
        number_of_test_examples = 10000                    # Max test observations
    # -------------------------------------------------------------------------
# Make runs repeatable
random_seed = 1
torch.backends.cudnn.enabled = False  # Disable cuDNN use of nondeterministic algorithms
torch.manual_seed(random_seed)
from optuna.visualization import plot_optimization_history, plot_slice, plot_contour
import matplotlib.pyplot as plt
import plotly.io as pi
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=number_of_trials, callbacks=[clbk])


In [ ]:
import sklearn
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])
# Save results to csv file
df = study.trials_dataframe().drop(['datetime_start', 'datetime_complete', 'duration'], axis=1)  # Exclude columns
df = df.loc[df['state'] == 'COMPLETE']        # Keep only results that did not prune
df = df.drop('state', axis=1)                 # Exclude state column
df = df.sort_values('value')                  # Sort based on accuracy
df.to_csv('optuna_results1.csv', index=False)  # Save to csv file
most_important_parameters = optuna.importance.get_param_importances(study, target=None)
print(most_important_parameters)
for key, value in most_important_parameters.items():
        print('  {}:{}{:.2f}%'.format(key, (15-len(key))*' ', value*100))

In [ ]:
from optuna.visualization import plot_optimization_history, plot_slice, plot_contour
import matplotlib.pyplot as plt
import plotly.io as pi
obj=plot_optimization_history(study)
obj.show()
# plt.savefig('OptHist.png')

pi.write_image(obj, 'OptHist.png')

obj=plot_slice(study)
obj.show()
pi.write_image(obj, 'OptHist.png')


In [ ]:
obj=plot_contour(study)
obj.show()

In [ ]:
# Generator set up

import time
import aotools
from radial import radial_data
import numpy as np
from scipy import fftpack
from astropy.io import fits
from soapy import SCI, confParse
from matplotlib import pyplot as plt
import os
import sys
# Small example of network outputs

# # Global import
# import sys
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# from torchvision import transforms
# from collections import OrderedDict

# Local import
# sys.path.insert(0, '../pytorch/models/')
# from Unet import UNet

sys.path.insert(0, '../src/pytorch/')
from dataset import *
# ------------------------------------------------------------------------
# Generate Point Spread functions from randomly drawn non-common path
# aberrations. The aberration follows a 1/f^2 law.
# One PSF in focus as well as a PSF out of focus are saved in FITS format
# (see astropy docs). The corresponding phase and Zernike Coefficient
# are also saved.
# ------------------------------------------------------------------------
os.system('rm -r Data2')
os.mkdir('Data2')
out='Data2'
np.random.seed(seed=0)
# sys.path.insert(0, '../src/pytorch/')
SOAPY_CONF = "psf.yaml"                         # Soapy config
# SOAPY_CONF = "psf.yaml"                         # Soapy config
gridsize = 128                                  # Pixel size of science camera
wavelength = 0.8e-6                             # Observational wavelength
diameter = 10                                   # Telescope diameter
pixelScale = 0.01                               # [''/px]s
WFE=3
n_psfs = 15                                     # Number of PSFs
n_zernike = 100                                # Number of Zernike polynomials
# Number of PSFs
                            # Number of Zernike polynomials
i_zernike = np.arange(2, n_zernike + 2)         # Zernike polynomial indices (piston excluded)
o_zernike= []                                   # Zernike polynomial radial Order, see J. Noll paper :
for i in range(1,n_zernike):                    # "Zernike polynomials and atmospheric turbulence", 1975
    for j in range(i+1):
        if len(o_zernike) < n_zernike:
            o_zernike.append(i)

# Generate randomly Zernike coefficient. By dividing the value
# by its radial order we produce a distribution following
# the expected 1/f^-2 law.
c_zernike = 2 * np.random.random((100*n_psfs, n_zernike)) - 1
for j in range(10*n_psfs):
    for i in range(n_zernike):
        c_zernike[j, i] = c_zernike[j, i] / o_zernike[i]
c_zernike = np.array([c_zernike[k, :] / np.abs(c_zernike[k, :]).sum()
                      * wavelength*(10**9) for k in range(100*n_psfs)])

# Update scientific camera parameters
config = confParse.loadSoapyConfig(SOAPY_CONF)
config.scis[0].pxlScale = pixelScale
config.tel.telDiam = diameter
config.calcParams()

mask = aotools.circle(config.sim.pupilSize / 2., config.sim.simSize).astype(np.float64)
zernike_basis = aotools.zernikeArray(n_zernike + 1, config.sim.pupilSize, norm='rms')

psfObj = SCI.PSF(config, nSci=0, mask=mask)

psfs_in = np.zeros((n_psfs, psfObj.detector.shape[0], psfObj.detector.shape[1]))
psfs_out = np.zeros((n_psfs, psfObj.detector.shape[0], psfObj.detector.shape[1]))

defocus = (wavelength / 4) * (10 ** 9) * zernike_basis[3, :, :]

t0 = time.time()
n_fail = 0
vals=0
i=0
# for i in range(n_pxsfs):
while vals<n_psfs:
#     WFE=np.range(3, 5, 100)
    WFE=np.random.randint(3,6, 1)
    aberrations_in = np.squeeze(np.sum(c_zernike[i, :, None, None] * WFE*zernike_basis[1:, :, :], axis=0))
    psfs_in[vals, :, :] = np.copy(psfObj.frame(aberrations_in.astype(np.float32)))


    # psfs_in[i, :, :] = np.random.poisson(lam=100000*psfs_in[i, :, :], size=None)
    # psfs_out[i, :, :] = np.random.poisson(lam=100000*psfs_out[i, :, :], size=None)
    
    # Save
    if (not np.isnan(psfs_in[vals, :, :]).any()):
       print('Eg')
       outfile = out+"/psf_" + str(vals) + ".fits"
       hdu_primary = fits.PrimaryHDU(c_zernike[i, :].astype(np.float32))
       hdu_phase = fits.ImageHDU(aberrations_in.astype(np.float32), name='PHASE')
       hdu_In = fits.ImageHDU(psfs_in[vals, :, :].astype(np.float32), name='INFOCUS')
       hdu_Out = fits.ImageHDU(psfs_in[vals, :, :].astype(np.float32), name='OUTFOCUS')
       hdu = fits.HDUList([hdu_primary, hdu_phase, hdu_In, hdu_Out])
       hdu.writeto(outfile, overwrite=True)
       vals=vals+1
    t_soapy = time.time() - t0
    i=i+1
    print(i)
print(vals)
print('Propagation and saving finished in {0:2f}s'.format(t_soapy))
print('Failed: {0:2f}'.format(n_fail))

In [ ]:
import optuna as op


In [ ]:
# Load validation set
import os
data_dir = 'Data1/'
print(os.listdir(data_dir))
dataset_size = 10
dataset = psf_dataset(root_dir = data_dir, 
                      size = dataset_size,
                      transform = transforms.Compose([Normalize(), ToTensor()]))

# Select one input (here id=0)

import matplotlib.pyplot as plt


for id in range(dataset_size): 
    sample = dataset[id]
    phase = sample['phase']
    image_in = sample['image'][0]
    image_out = sample['image'][1]

    f, axarr = plt.subplots(1, 3, figsize=(15, 10))
    im1 = axarr[0].imshow(phase, cmap=plt.cm.jet)
#     im1.set_clim(-np.pi, np.pi)
    axarr[0].set_title("Phase")
    plt.colorbar(im1, ax = axarr[0], fraction=0.046)
    im2 = axarr[1].imshow(image_in, cmap=plt.cm.jet)
    axarr[1].set_title("In")
    plt.colorbar(im2, ax = axarr[1], fraction=0.046)
    im3 = axarr[2].imshow(image_out, cmap=plt.cm.jet)
    axarr[2].set_title("Out")
    plt.colorbar(im3, ax = axarr[2], fraction=0.046)
    plt.show()

In [ ]:
import json
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
# sys.path.insert(0, '../src/pytorch/models/')
# print(os.listdir('../src/pytorch/models/'))
dls='NewOne4/metrics.json'

# dls='mts.json'
with open(dls) as f:
    file= json.load(f)
    trainLoss=file['train_loss']
    valLoss= file['val_loss']
    lr=file['learning_rate']
#     print(json.load(f))
#     print(d)
Nmbers=np.arange(0, len(trainLoss))
plt.scatter(Nmbers, trainLoss)
plt.ylabel('Training Loss')
plt.xlabel('Epochs')
plt.ylim(0., 0.05)
plt.show()
Nmbers=np.arange(0, len(valLoss))
# print(valLoss)
plt.scatter(Nmbers, valLoss)
plt.ylabel('Valuation Loss')
plt.xlabel('Epochs')
# plt.xlim(50, 300)
# plt.ylim(0.05,0.13)
plt.show()

In [ ]:
#LOAD DATA 

sample=dataset[0]
print(sample['image'].unsqueeze(0))
def shrink(data, rows, cols):
    shrunk = np.zeros((rows,cols))
    for i in list(range(0,rows)):
        for j in list(range(0,cols)):
            row_sp = int(data.shape[0]/rows)
            col_sp = int(data.shape[1]/cols)
#             print(col_sp, row_sp)
            zz = data[i*row_sp : i*row_sp + row_sp, j*col_sp : j*col_sp + col_sp]
            shrunk[i,j] = np.sum(zz)
    return shrunk
im2=plt.imread('../../../../../CNN/ML/src/generation/ar1/run06/Shot22.tif').astype(float)
im22=np.loadtxt('../../../../../CNN/ML/src/generation/arNew/ar/sphr33.txt')
data=im2
sample=dataset[i]
phase=sample['phase']
input = sample['image'].unsqueeze(0)
print(np.shape(input[0][0]))
plt.imshow(im2[ 900:1700,1200:2000])
fin=(data - data.mean()) / (data.max() - data.min())
fin=shrink(fin[ 900:1796,1200:2096], 128, 128)
input = torch.from_numpy(np.array([[fin, fin]]))

print(input)
output = model(input)
output_numpy = output.detach().numpy()[0][0]
phase_est = output_numpy

#     print(phase)
# Plot network output
# f, axarr = plt.subplots(1, 3, figsize=(15, 10))
# im1 = axarr[0].imshow(phase, cmap=plt.cm.jet)
# minlim = torch.min(phase)
# maxlim=torch.max(phase)
# axarr[0].set_title("Phase exact")
plt.colorbar(im1, ax = axarr[0], fraction=0.046)
im2 = axarr[1].imshow(output_numpy, cmap=plt.cm.jet)
im2.set_clim(minlim, maxlim)
axarr[1].set_title("Phase estimated ")
plt.colorbar(im2, ax = axarr[1], fraction=0.046)
im3 = axarr[2].imshow(phase.numpy()-output_numpy, cmap=plt.cm.jet)
axarr[2].set_title("Residual (Exact-Estimated)")
plt.colorbar(im3, ax = axarr[2], fraction=0.046)
plt.show()

In [ ]:
# Load model architectures and weights

model = UNet(2, 1)
# sample=dataset[i]
model_dir = 'LR2/model.pth'
# model_dir='/Users/jh2619/Desktop/Machine-learning-for-image-based-wavefront-sensing-master/src/pytorch/models/Unet.py'
state_dict = torch.load(model_dir, map_location='cpu')
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:] # remove module.
    new_state_dict[name] = v
model.load_state_dict(state_dict)

In [ ]:
# Network prediction
vls=[]
# Forward pass and get phase estimation
# Need to expand firts dimension to match batch format
# [batch]x[channel]x[dim_x]x[dim_y]
ids=[0,1,2,3,4]
for i in ids:
    sample=dataset[i]
    phase=sample['phase']
    input = sample['image'].unsqueeze(0)
    output = model(input)
    output_numpy = output.detach().numpy()[0][0]
    phase_est = output_numpy
#     print(phase)
    # Plot network output
    f, axarr = plt.subplots(1, 3, figsize=(15, 10))
    im1 = axarr[0].imshow(phase, cmap=plt.cm.jet)
    minlim = torch.min(phase)
    maxlim=torch.max(phase)
    axarr[0].set_title("Phase exact")
    plt.colorbar(im1, ax = axarr[0], fraction=0.046)
    im2 = axarr[1].imshow(output_numpy, cmap=plt.cm.jet)
    im2.set_clim(minlim, maxlim)
    axarr[1].set_title("Phase estimated ")
    plt.colorbar(im2, ax = axarr[1], fraction=0.046)
    im3 = axarr[2].imshow(phase.numpy()-output_numpy, cmap=plt.cm.jet)
    axarr[2].set_title("Residual (Exact-Estimated)")
    plt.colorbar(im3, ax = axarr[2], fraction=0.046)
    plt.show()
    vls.append(np.sum(abs(phase.numpy()-output_numpy)))
print(np.mean(vls))

In [ ]:
# Project phase on Zernike Basis

# 1) Define the basis
z_basis = aotools.zernikeArray(101, 128, norm='rms')  # [rad]

# 2) Define Standard Dot product
def frobeniusInnerProduct(A, B):
    fIProduct = 0
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            fIProduct += A[i, j]*B[i, j]
    return fIProduct;

# 3) Project
z_norm = np.empty([100])
z_coeffs = np.empty([100])
z_coeffs_est = np.empty([100])
for i in range(100):
    z_norm[i] = frobeniusInnerProduct(z_basis[i+1,:,:], z_basis[i+1,:,:])
    z_coeffs[i] = frobeniusInnerProduct(phase, z_basis[i+1,:,:]) / z_norm[i]
    z_coeffs_est[i] = frobeniusInnerProduct(phase_est, z_basis[i+1,:,:]) / z_norm[i]
    
# 4) Plot
colors2 = ['#669f61','#ff884d','#32526e','#81b9c3','#41c3ac']
nbr = 30;
i_zernike = np.arange(2, nbr+2)
f, ax = plt.subplots(2, 1, figsize=(15, 8))
w = 0.4
ax[0].bar(i_zernike-w/2, np.abs(z_coeffs[0:nbr]),  width=w, label='Original', color=colors2[2],zorder=3)
ax[0].bar(i_zernike +w/2, np.abs(z_coeffs_est[0:nbr]),  width=w, label='Estimated', color=colors2[3],zorder=3)
ax[0].set_ylabel('Absolute magnitude [Rad]')
ax[0].legend(fancybox=False, frameon=False)
ax[1].bar(i_zernike-w/2, np.abs(z_coeffs[nbr:2*nbr]),  width=w, label='Original', color=colors2[2],zorder=3)
ax[1].bar(i_zernike +w/2, np.abs(z_coeffs_est[nbr:2*nbr]), width=w, label='Estimated', color=colors2[3],zorder=3)
ax[1].set_ylabel('Absolute magnitude [Rad]')
ax[1].set_xlabel("Zernike indices")
ax[1].set_xticks(np.arange(2, nbr+2)[0::4])
ax[1].set_xticklabels(np.arange(nbr+2, nbr+nbr+2)[0::4])
ax[0].set_ylim(0,0.45)
ax[1].set_ylim(0,0.45)
ax[0].grid(zorder=0, color='lightgray', linestyle='--')
ax[1].grid(zorder=0, color='lightgray', linestyle='--')
plt.show() 

In [ ]:
sys.path.insert(0, '../src/algorithms/')
from utils import *

# Wavefront root mean squared error over the phase map (pixel wise) [Rad]
rmse_wfe = rootMeanSquaredError(phase, phase_est).numpy()
print("WFE RMSE: ", rmse_wfe, "[Rad]")
print("WFE RMSE: ", 2200*rmse_wfe/(2*np.pi), "[nm]")

# Wavefront root mean squared error (wfe rmse) [Rad]
rmse_zernike = rootMeanSquaredError(z_coeffs, z_coeffs_est, False)
print("Zernike RMSE: ", rmse_zernike, "[Rad]")
print("Zernike RMSE: ", 2200*rmse_zernike/(2*np.pi), "[nm]")